In [1]:
import tensorflow as tf
import keras

from keras.engine import  Model
from keras.layers import Flatten, Dense, Input
from keras_vggface.vggface import VGGFace
from keras import backend as K
from custom_metrics import f1, precision, recall

Using TensorFlow backend.


In [2]:
config = tf.ConfigProto( device_count = {'GPU': 1 } ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

In [4]:
#custom parameters
nb_class = 225
hidden_dim = 128

vgg_model = VGGFace(include_top=False, input_shape=(224, 224, 3))
last_layer = vgg_model.get_layer('pool5').output
x = Flatten(name='flatten')(last_layer)
x = Dense(hidden_dim, activation='relu', name='fc6')(x)
x = Dense(hidden_dim, activation='relu', name='fc7')(x)
out = Dense(nb_class, activation='softmax', name='fc8')(x)
custom_vgg_model = Model(vgg_model.input, out)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [6]:
for layer in custom_vgg_model.layers[:-3]:
    layer.trainable = False

In [7]:
from keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255 as we need to normalize them before feeding to the network
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

# We can also use train_datagen.flow(X, label_X) but with flow_from_directory, the classname will be each directory
# Flow training images in batches of 64 using train_datagen generator

train_generator = train_datagen.flow_from_directory(
        './Data/train/',  # This is the source directory for training images
        target_size=(224, 224),  # All images will be resized to 150x150
        batch_size=64,
        class_mode='categorical',
        shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
        './Data/validation',  # This is the source directory for training images
        target_size=(224, 224),  # All images will be resized to 150x150
        batch_size=64,
        class_mode='categorical',
        shuffle=True)

Found 30079 images belonging to 225 classes.
Found 5177 images belonging to 225 classes.


In [9]:
from keras.callbacks import TensorBoard
custom_vgg_model.compile(optimizer=keras.optimizers.Adam(lr=0.0002), loss='categorical_crossentropy',metrics=['accuracy', f1, precision, recall])
NAME = 'Farzad-2Dense-255class-VGGFACE-correctedData-farzad-new'

filepath= NAME + "-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1,save_weights_only=False, save_best_only=True, mode='max')
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))
callbacks_list = [checkpoint, tensorboard]

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [ ]:
custom_vgg_model.fit_generator(train_generator,
      steps_per_epoch=train_generator.samples//64,  
      epochs=20,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=validation_generator.samples//64,
      callbacks =callbacks_list
    )

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
470/469 [==============================] - 305s 649ms/step - loss: 3.5153 - acc: 0.3984 - f1: 0.1462 - precision: 0.5945 - recall: 0.0893 - val_loss: 1.8316 - val_acc: 0.6969 - val_f1: 0.4727 - val_precision: 0.9855 - val_recall: 0.3133
Epoch 2/20
470/469 [==============================] - 293s 623ms/step - loss: 1.1903 - acc: 0.8006 - f1: 0.6780 - precision: 0.9817 - recall: 0.5262 - val_loss: 0.8873 - val_acc: 0.8368 - val_f1: 0.7749 - val_precision: 0.9740 - val_recall: 0.6454
Epoch 3/20
470/469 [==============================] - 293s 623ms/step - loss: 0.6320 - acc: 0.8925 - f1: 0.8434 - precision: 0.9816 - recall: 0.7416 - val_loss: 0.6228 - val_acc: 0.8818 - val_f1: 0.8528 - val_precision: 0.9749 - val_recall: 0.7599
Epoch 4/20
470/469 [==============================] - 293s 623ms/step - loss: 0.4227 - acc: 0.9277 - f1: 0.9006 - precision: 0.9837 - recall: 0.8319 - val_loss: 0.5073 - val_acc: 0.8976 - val_f1: 0.8867 - val